In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import sys
import shutil
import math
import random
import heapq 
import time
from PIL import Image
from io import StringIO,BytesIO 
import cv2
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from functools import reduce
import wfdb#https://github.com/MIT-LCP/wfdb-python
from wfdb import processing
import faiss 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
torch.cuda.set_device(0)
print (torch.cuda.current_device())

Loading faiss with AVX2 support.


0


In [4]:
def log_gaussian_loss(output, target, sigma, no_dim, sum_reduce=True):
    exponent = -0.5*(target - output)**2/sigma**2
    log_coeff = -no_dim*torch.log(sigma) - 0.5*no_dim*np.log(2*np.pi)
    
    if sum_reduce:
        return -(log_coeff + exponent).sum()
    else:
        return -(log_coeff + exponent)
    
class gaussian:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
        
    def loglik(self, weights):
        exponent = -0.5*(weights - self.mu)**2/self.sigma**2
        log_coeff = -0.5*(np.log(2*np.pi) + 2*np.log(self.sigma))
        
        return (exponent + log_coeff).sum()
    
class BayesLinear_Normalq(nn.Module):
    def __init__(self, input_dim, output_dim, prior):
        super(BayesLinear_Normalq, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.prior = prior
        
        self.weight_mus = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-0.01, 0.01))
        self.weight_rhos = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-3, -3))
        
    def forward(self, x):
        # sample gaussian noise for each weight
        weight_epsilons = Variable(self.weight_mus.data.new(self.weight_mus.size()).normal_())      
        # calculate the weight stds from the rho parameters
        weight_stds = torch.log(1 + torch.exp(self.weight_rhos))
        # calculate samples from the posterior from the sampled noise and mus/stds
        weight_sample = self.weight_mus + weight_epsilons*weight_stds
    
        output = torch.mm(x, weight_sample)
            
        # computing the KL loss term
        #reference: https://github.com/jojonki/AutoEncoders/blob/master/kl_divergence_between_two_gaussians.pdf
        prior_cov, varpost_cov = self.prior.sigma**2, weight_stds**2
        KL_loss = 0.5*(torch.log(prior_cov/varpost_cov)).sum() - 0.5*weight_stds.numel()
        KL_loss = KL_loss + 0.5*(varpost_cov/prior_cov).sum()
        KL_loss = KL_loss + 0.5*((self.weight_mus - self.prior.mu)**2/prior_cov).sum()
            
        return output, KL_loss


class BayesianNeuralNetwork(nn.Module):
    def __init__(self, input_dim=50, num_units=[128,256], output_dim=8):
        super(BayesianNeuralNetwork, self).__init__()
        
        # network with Bayesian linear.
        self.layer1 = BayesLinear_Normalq(input_dim, num_units[0], gaussian(0, 3))
        self.layer2 = BayesLinear_Normalq(num_units[0], num_units[1], gaussian(0, 3))
        self.layer3 = BayesLinear_Normalq(num_units[1], output_dim, gaussian(0, 3))
        
        # activation to be used between hidden layers
        self.activation = nn.ReLU(inplace = True)
        # noise
        self.log_noise = nn.Parameter(torch.cuda.FloatTensor([3]))
    
    def forward(self, x):
        
        KL_loss_total = 0
        #x = x.view(-1, self.input_dim)
        x = x.view(x.size(0),-1) 
        #layer1
        x, KL_loss = self.layer1(x)
        x = self.activation(x)
        KL_loss_total = KL_loss_total + KL_loss
        #layer2
        x, KL_loss = self.layer2(x)
        x = self.activation(x) 
        KL_loss_total = KL_loss_total + KL_loss
        #layer3
        out, KL_loss = self.layer3(x)
        KL_loss_total = KL_loss_total + KL_loss
        
        return x, out, KL_loss_total


x1 = torch.rand(10,50).cuda()
y = torch.LongTensor([0,1,2,3,4,5,6,7,0,1]).cuda()
model = BayesianNeuralNetwork().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
for epoch in range(10):
    optimizer.zero_grad()
    
    _,out,KL_loss = model(x1)
    out = F.log_softmax(out)
    fit_loss = F.nll_loss(out, y)
    loss =  (KL_loss+fit_loss)/len(y)
    loss.backward()
    optimizer.step()
    sys.stdout.write('\r {} : loss = {}'.format(epoch, float('%0.6f'%loss.item())))
    #sys.stdout.flush()
#output
x2 = torch.rand(2,50).cuda()
x2,_,_ = model(x2)
#print (x2)
print (x2.size())

 9 : loss = 14897.131836torch.Size([2, 256])


/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:93: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
